In [11]:
import numpy as np, pandas as pd

def inverse_great_circle(r1, dec1, sep, pos_angle):
    """
    Calculate the position of a point on a great circle given the position of another point, 
    the angular separation between the two points, and the position angle of the second point 
    relative to the great circle connecting the two points.

    Parameters
    ----------
    r1, dec1 : float or `~numpy.ndarray`
        The right ascension and declination of the first point in degrees.
    sep, pos_angle : float or `~numpy.ndarray`
        The angular separation between the two points in radians and the position angle of the 
        second point relative to the great circle connecting the two points in radians.

    Returns
    -------
    r2, dec2 : float or `~numpy.ndarray`
        The right ascension and declination of the second point in degrees.

    """

    r1, dec1, sep, pos_angle = np.broadcast_arrays(r1, dec1, sep, pos_angle)

    #pos_angle = np.deg2rad(pos_angle)
    dec1 = np.deg2rad(dec1)
    r1 = np.deg2rad(r1)
    #sep = np.deg2rad(sep)

    dec2 = np.arcsin(np.sin(dec1) * np.cos(sep) + np.cos(dec1) * np.sin(sep) * np.cos(pos_angle))
    r2 = r1 + np.arctan2(np.sin(pos_angle) * np.sin(sep) * np.cos(dec1),
                         np.cos(sep) - np.sin(dec1) * np.sin(dec2))

    r2 = np.rad2deg(r2) % 360
    dec2 = np.rad2deg(dec2)

    return r2, dec2

def celestial_ellipse(ra, dec, r0, r1, pa, num_point=200):
    
    angles = np.linspace(0, 2.*np.pi,  num_point)
    
    pa_rad = pa*np.pi/180
    
    ang_dis = 1./np.sqrt((np.cos(angles-pa_rad)/(r0*np.pi/180.))**2+(np.sin(angles-pa_rad)/(r1*np.pi/180.))**2)
    
    r2s, dec2s = inverse_great_circle(np.array([ra]*num_point), np.array([dec]*num_point), ang_dis, angles)
    
    return r2s, dec2s




In [51]:
fgl = pd.read_csv('/Users/huiyang/Research/GitHub/FGL_Class/4fgl_dr4/gll_psc_v32.csv')
fgl['CLASS1'] = fgl['CLASS1'].fillna('')
fgl['CLASS2'] = fgl['CLASS2'].fillna('')
# print(fgl[['CLASS1','CLASS2']].value_counts())
fgl['FGL'] = fgl['Source_Name'].str[5:]
fgl['name_type'] = ''
fgl.loc[fgl['Source_Name'].str[-1]=='e','name_type'] = 'e'
fgl.loc[fgl['Source_Name'].str[-1]=='c','name_type'] = 'c'
print(fgl['name_type'].value_counts())
fgl['CLASS_g'] = 'UA'
fgl.loc[fgl['CLASS1'].str.isupper(),'CLASS_g'] = 'ID'
fgl.loc[(fgl['CLASS1']!='') & (fgl['CLASS_g']!='ID'),'CLASS_g'] = 'AS'
# print(fgl['CLASS_g'].value_counts())
# print(fgl['CLASS_g'].value_counts(normalize=True))
# print(len(fgl[(fgl['CLASS_g']=='UA') & (abs(fgl['GLAT'])<=5)]))
# print(len(fgl[(fgl['CLASS_g']=='UA') & (abs(fgl['GLAT'])<=5)])/len(fgl[(abs(fgl['GLAT'])<=5)]))
# print(len(fgl[((fgl['CLASS_g']=='UA') | (fgl['CLASS_g']=='AS')) & (abs(fgl['GLAT'])<=3.5)]))
# print(len(fgl[((fgl['CLASS_g']=='UA') | (fgl['CLASS_g']=='AS')) & (abs(fgl['GLAT'])<=3.5)])/len(fgl[(abs(fgl['GLAT'])<=3.5)]))
# print('---------')

# fgl_G = fgl[(fgl['GLAT']>=-90.) & (fgl['GLAT']<=90.)].reset_index(drop=True)

# fgl_unas = fgl[(fgl['CLASS1']=='') | (fgl['CLASS1']=='unk') | (fgl['CLASS1']=='spp') | (fgl['CLASS1']=='pwn') | (fgl['CLASS1']=='snr')].reset_index(drop=True)
# print(len(fgl_unas))
# print(fgl_unas['CLASS1'].value_counts())
# print(fgl_unas[['CLASS1','name_type']].value_counts())

fgl.loc[fgl['CLASS_g']=='ID','CLASS1'].value_counts()

# df_idf = fgl[fgl['CLASS_g']=='ID'].reset_index(drop=True)
# print(len(df_idf['CLASS1']))

df_idf = pd.DataFrame()
for clas in ['PSR','MSP','PWN','SNR','SPP','FSRQ' ,'BLL','AGN','NLSY1','RDG','BCU','GAL','HMB','LMB','BIN','NOV','SFR','GC']:
    df_idf = pd.concat([df_idf, fgl[fgl['CLASS1']==clas]], ignore_index=True, sort=False)

print(len(df_idf['CLASS1']))

# df_idf['CLASS1'] = df_idf['CLASS1'].replace({
#               "BCU":"AGN","BIN":"Other","BLL":"AGN","FSRQ":"AGN",
#                "GAL":"Other","GC":"Other","HMB":"Other","LMB":"Other","MSP":"PSR","NLSY1":"AGN","NOV":"Other","PWN":"SPP",
#               "RDG":"AGN","SFR":"Other","SNR":"SPP","CSS":"AGN","GLC":"Other","SBG":"Other","SEY":"AGN","SPP":"SPP","SSRQ":"AGN","unk":'Unassc'})

fgl_ext = pd.read_csv('/Users/huiyang/Research/GitHub/FGL_Class/4fgl_dr4/gll_psc_v32_extend.csv')

for ext_src in df_idf.loc[df_idf['Source_Name'].str[-1]=='e', 'Extended_Source_Name']:
    #print(ext_src)
    
    df_idf.loc[df_idf['Extended_Source_Name']==ext_src, 'Conf_95_SemiMajor',] = fgl_ext.loc[fgl_ext['Source_Name']==ext_src, 'Model_SemiMajor'].values[0]
    df_idf.loc[df_idf['Extended_Source_Name']==ext_src, 'Conf_95_SemiMinor'] = fgl_ext.loc[fgl_ext['Source_Name']==ext_src, 'Model_SemiMinor'].values[0]
    df_idf.loc[df_idf['Extended_Source_Name']==ext_src,'Conf_95_PosAng'] = fgl_ext.loc[fgl_ext['Source_Name']==ext_src,'Model_PosAng'].values[0]
    
df_idf = df_idf[~df_idf['Conf_95_SemiMajor'].isnull()].reset_index(drop=True)
df_idf['Conf_95_SemiMajor'] = df_idf['Conf_95_SemiMajor'].fillna(0.1)
df_idf['Conf_95_SemiMinor'] = df_idf['Conf_95_SemiMinor'].fillna(0.1)
df_idf['Conf_95_PosAng'] = df_idf['Conf_95_PosAng'].fillna(0)
print(df_idf['CLASS1'].value_counts())

# df_idf = df_idf.sample(n=150).reset_index(drop=True)
# print(df_idf['CLASS1'].value_counts())

     6763
c     351
e      81
Name: name_type, dtype: int64
423
MSP      137
PSR      136
FSRQ      44
SNR       26
BLL       22
PWN       11
HMB        8
RDG        6
SPP        6
NLSY1      4
NOV        4
SFR        3
GAL        2
LMB        2
AGN        1
BCU        1
BIN        1
GC         1
Name: CLASS1, dtype: int64


In [55]:

#num=3
for FGL, ra, dec, r0, r1, PA in zip(df_idf['FGL'].values, df_idf['RAJ2000'].values, df_idf['DEJ2000'].values, df_idf['Conf_95_SemiMajor'].values, df_idf['Conf_95_SemiMinor'].values, df_idf['Conf_95_PosAng'].values):
    #print(FGL, ra, dec, r0, r1, PA)
    ra2s, dec2s = celestial_ellipse(ra, dec, r0, r1, PA)
    #print(ra2s,dec2s)
    arys = f'{[[ra2s[i], dec2s[i]] for i in range(len(ra2s))]}'
    # print(f'            A.polygon({arys}),')
    # print(f'                {{name:"{FGL}", coord:"{ra} {dec}"}},')
    
    df_fgl_src = df_idf[df_idf['FGL']==FGL].reset_index(drop=True)
    Gname, GLON, GLAT, G100, Gsig, Var_idx, sig_curv = df_fgl_src.loc[0, 'Source_Name'], df_fgl_src.loc[0, 'GLON'], df_fgl_src.loc[0, 'GLAT'], df_fgl_src.loc[0, 'Energy_Flux100'], df_fgl_src.loc[0, 'Signif_Avg'], df_fgl_src.loc[0, 'Variability_Index'], df_fgl_src.loc[0, 'LP_SigCurv']
    CLAS1,ASSOC1 = df_fgl_src.loc[0, 'CLASS1'],df_fgl_src.loc[0, 'ASSOC1']
    print(f"              case '{ra} {dec}':")
    print(f'                g_prop1.textContent ="Name={Gname},GLON={GLON:.1f},GLAT={GLAT:.1f},Class={CLAS1},{ASSOC1}";')
    print(f'                g_prop2.textContent ="Signif={Gsig:.1f},EnergyFlux100={G100:.1e}cgs,Var_idx={Var_idx:.1f},Sig_Cur={sig_curv:.1f}";')
    print(f'              break;')





              case '0.7201 62.2905':
                g_prop1.textContent ="Name=4FGL J0002.8+6217,GLON=117.3,GLAT=-0.1,Class=PSR,PSR J0002+6216";
                g_prop2.textContent ="Signif=35.8,EnergyFlux100=1.9e-11cgs,Var_idx=15.4,Sig_Cur=12.0";
              break;
              case '1.768 73.0512':
                g_prop1.textContent ="Name=4FGL J0007.0+7303,GLON=119.7,GLAT=10.5,Class=PSR,PSR J0007+7303";
                g_prop2.textContent ="Signif=401.7,EnergyFlux100=4.3e-10cgs,Var_idx=8.2,Sig_Cur=82.9";
              break;
              case '16.6094 48.9269':
                g_prop1.textContent ="Name=4FGL J0106.4+4855,GLON=125.5,GLAT=-13.9,Class=PSR,PSR J0106+4855";
                g_prop2.textContent ="Signif=65.7,EnergyFlux100=2.0e-11cgs,Var_idx=6.3,Sig_Cur=17.7";
              break;
              case '31.4273 64.8239':
                g_prop1.textContent ="Name=4FGL J0205.7+6449,GLON=130.7,GLAT=3.1,Class=PSR,PSR J0205+6449";
                g_prop2.textContent ="Signif

In [39]:
df_idf.loc[df_idf['Conf_95_SemiMajor'].isnull(), ['FGL','CLASS1']]

,FGL,CLASS1
80,J1731.7-4744,PSR
242,J1909.7-3744,MSP
276,J0534.5+2201i,PWN
277,J0534.5+2201s,PWN
411,J0358.4-5446,NOV
415,J1820.8-2822,NOV
417,J2023.5+2046,NOV
418,J2102.1+4546,NOV


In [58]:
from astropy.table import Table
from astropy.io.votable import from_table

df_ass = df_idf[['ASSOC1','RA_Counterpart','DEC_Counterpart','Unc_Counterpart']]
df_ass['PU'] = df_ass['Unc_Counterpart']*3600
df_ass = df_ass.rename(columns={'RA_Counterpart':'ra','DEC_Counterpart':'dec'})
votable = from_table(Table.from_pandas(df_ass[['ASSOC1','ra','dec','PU']]))
votable.to_xml("/Users/huiyang/Research/GitHub/FGL-aladin-lite-test/FGL_IDF.vot")

/var/folders/ty/s6dnp14d37b0_qcxbb5frflw0000gn/T/ipykernel_80563/2425460557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ass['PU'] = df_ass['Unc_Counterpart']*3600
